In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('/Users/gabriel/Documents/GitHub/Groupwork-DataAnalysis/California Wildfire Damage2.csv')
df

,Incident_ID,Date,Location,Area_Burned (Acres),Homes_Destroyed,Businesses_Destroyed,Vehicles_Damaged,Injuries,Fatalities,Estimated_Financial_Loss (Million $),Cause
0,INC1000,2020-11-22,Sonoma County,14048,763,474,235,70,19,2270.57,Lightning
1,INC1001,2021-09-23,Sonoma County,33667,1633,4,263,100,2,1381.14,Lightning
2,INC1002,2022-02-10,Shasta County,26394,915,291,31,50,6,2421.96,Human Activity
3,INC1003,2021-05-17,Sonoma County,20004,1220,128,34,28,0,3964.16,Unknown
4,INC1004,2021-09-22,Sonoma County,40320,794,469,147,0,15,1800.09,Unknown
...,...,...,...,...,...,...,...,...,...,...,...
95,INC1095,2018-01-16,Mendocino County,13112,1468,46,281,89,17,4008.58,Human Activity
96,INC1096,2022-07-07,Shasta County,39209,710,282,189,34,14,376.72,Human Activity
97,INC1097,2016-05-18,Shasta County,11863,702,9,40,100,1,1474.78,Human Activity
98,INC1098,2014-06-24,Napa Valley,1338,1619,262,293,94,3,3308.74,Human Activity


In [3]:
df.count()

Incident_ID                             100
Date                                    100
Location                                100
Area_Burned (Acres)                     100
Homes_Destroyed                         100
Businesses_Destroyed                    100
Vehicles_Damaged                        100
Injuries                                100
Fatalities                              100
Estimated_Financial_Loss (Million $)    100
Cause                                   100
dtype: int64

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 11 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Incident_ID                           100 non-null    object 
 1   Date                                  100 non-null    object 
 2   Location                              100 non-null    object 
 3   Area_Burned (Acres)                   100 non-null    int64  
 4   Homes_Destroyed                       100 non-null    int64  
 5   Businesses_Destroyed                  100 non-null    int64  
 6   Vehicles_Damaged                      100 non-null    int64  
 7   Injuries                              100 non-null    int64  
 8   Fatalities                            100 non-null    int64  
 9   Estimated_Financial_Loss (Million $)  100 non-null    float64
 10  Cause                                 100 non-null    object 
dtypes: float64(1), int64

In [5]:
df.describe()

,Area_Burned (Acres),Homes_Destroyed,Businesses_Destroyed,Vehicles_Damaged,Injuries,Fatalities,Estimated_Financial_Loss (Million $)
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,26531.460000,941.890000,251.570000,150.330000,42.040000,9.930000,2396.099300
std,14282.123824,543.019967,138.317761,88.471197,29.201467,5.682251,1430.439422
min,357.000000,18.000000,4.000000,5.000000,0.000000,0.000000,52.590000
25%,15916.250000,501.000000,134.750000,70.750000,16.000000,5.000000,1175.195000
50%,25618.000000,908.500000,256.500000,150.500000,37.000000,10.000000,2408.530000
75%,39775.000000,1401.750000,371.000000,229.750000,60.000000,14.250000,3662.110000
max,49653.000000,1968.000000,493.000000,300.000000,100.000000,20.000000,4866.990000


In [6]:
df.dropna(inplace=True)

In [7]:
print(df.isna().sum())  # Or df.isnull().sum() : it count the number of missing values per column

Incident_ID                             0
Date                                    0
Location                                0
Area_Burned (Acres)                     0
Homes_Destroyed                         0
Businesses_Destroyed                    0
Vehicles_Damaged                        0
Injuries                                0
Fatalities                              0
Estimated_Financial_Loss (Million $)    0
Cause                                   0
dtype: int64


I Think that we finished the cleaning of our dataset. There are no NaN or incoherent values, the columns names are understandable and evely columns have the same amount of datas with no missing values